In [5]:
    #preprocessing
    using Pkg
    using DataFrames
    using CSV
    using Plots
    using GLM
    using StatsBase
    using Lathe
    using MLBase
    using ClassImbalance
    using CPUTime
    using Lathe.preprocess: TrainTestSplit
    using BenchmarkTools

In [6]:
function log_rec()

    #number of columns
    ENV["COLUMNS"] = 1000

    #importing dataFrame 
    df = DataFrame(CSV.File("datahai.csv"))
    describe(df)
    names(df)

    #now finding number of  ClassImbalance
    countmap(df.Exited)
    names(df)
    #One hot encoding 
    Lathe.preprocess.OneHotEncode(df,:Geography)
    Lathe.preprocess.OneHotEncode(df,:Gender)
    select!(df, Not([:RowNumber, :CustomerId,:Surname,:Geography,:Gender,:Male]))
    first(df,5)
    names(df)
    #Train test split
    
    train , test = TrainTestSplit(df,0.8)
    train
    test

    #Modelling
    fm = @formula(Exited ~ CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain)
    logit = glm(fm , train , Binomial() ,ProbitLink())

    #prediction 
    prediction = predict(logit,test)

    #prediction class 
    prediction_class = [if x<0.5 0 else 1 end for x in prediction]

    prediction_df = DataFrame(y_actual = test.Exited, y_predicted = prediction_class, prob_predicted = prediction);
    prediction_df.correctly_classified = prediction_df.y_actual .== prediction_df.y_predicted


    accuracy = mean(prediction_df.correctly_classified)

end


log_rec (generic function with 1 method)

In [22]:
@time log_rec()

  0.027746 seconds (3.32 k allocations: 10.947 MiB)


0.8153452685421995

In [17]:
GC.gc()
@CPUtime begin
    log_rec()
end

elapsed CPU time: 0.025021 seconds


0.8179979777553084